In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

RAND_STATE = 32 # for reproducible shuffling
TT_RATIO = 0.25 # test/train

from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [5]:
data_berlin = pd.read_csv('data_berlin.csv')

In [6]:
data_berlin = data_berlin.drop(['Unnamed: 0'], axis=1)

In [7]:
from scipy import stats

# Assuming df is the name of your dataframe
z_scores = np.abs(stats.zscore(data_berlin))

# Remove rows where any of the columns have a z-score greater than 3
data_berlin_cleaned = data_berlin[(z_scores < 3).all(axis=1)]

In [8]:
data_berlin_cleaned.corr()

,realSum,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,lng,lat,room_shared_True,room_private_True,host_is_superhost_True,weekend_yes
realSum,1.000000,0.487413,0.015543,0.265337,0.042779,-0.042480,0.236580,-0.193626,-0.136797,0.355710,0.358252,-0.078368,0.104003,NaN,-0.585169,-0.026724,0.037382
person_capacity,0.487413,1.000000,0.039284,0.281573,-0.145774,-0.199715,0.329071,-0.050778,-0.011299,0.064578,0.081578,0.005789,0.058498,NaN,-0.413488,-0.076826,-0.022496
multi,0.015543,0.039284,1.000000,-0.268677,-0.017115,-0.005838,0.020453,-0.025337,-0.037347,0.019580,0.041898,0.036798,-0.032593,NaN,0.049235,0.088648,0.000058
biz,0.265337,0.281573,-0.268677,1.000000,-0.080403,-0.257225,-0.044848,-0.082156,-0.103710,0.188578,0.178370,-0.091410,0.059093,NaN,-0.201484,-0.121728,-0.034735
cleanliness_rating,0.042779,-0.145774,-0.017115,-0.080403,1.000000,0.572277,-0.012577,0.043850,-0.034789,-0.005152,-0.016364,-0.045061,-0.001484,NaN,0.010420,0.232036,-0.009343
guest_satisfaction_overall,-0.042480,-0.199715,-0.005838,-0.257225,0.572277,1.000000,0.025219,0.046631,-0.031919,-0.062042,-0.051565,-0.002416,-0.019224,NaN,0.074650,0.273766,0.005896
bedrooms,0.236580,0.329071,0.020453,-0.044848,-0.012577,0.025219,1.000000,0.036945,0.036486,-0.067687,-0.049911,-0.001697,0.013010,NaN,-0.072223,0.017410,-0.010679
dist,-0.193626,-0.050778,-0.025337,-0.082156,0.043850,0.046631,0.036945,1.000000,0.398814,-0.634495,-0.618186,-0.429330,-0.486321,NaN,0.049138,-0.001947,0.001399
metro_dist,-0.136797,-0.011299,-0.037347,-0.103710,-0.034789,-0.031919,0.036486,0.398814,1.000000,-0.392623,-0.431262,0.144460,-0.149153,NaN,-0.002419,-0.052452,0.001838
attr_index_norm,0.355710,0.064578,0.019580,0.188578,-0.005152,-0.062042,-0.067687,-0.634495,-0.392623,1.000000,0.867699,-0.057305,0.238015,NaN,-0.128405,0.012062,0.001162


In [13]:
data_berlin_cleaned_reg = data_berlin_cleaned.drop(['guest_satisfaction_overall', 'host_is_superhost_True', 'lng', 'lat'], axis=1)

In [14]:
x = data_berlin_cleaned_reg.drop('realSum', axis=1)
y = data_berlin_cleaned_reg.realSum

In [15]:
# test-train split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TT_RATIO, random_state=RAND_STATE)
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
x_train.describe()

,person_capacity,multi,biz,cleanliness_rating,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,room_shared_True,room_private_True,weekend_yes
count,1594.000000,1594.000000,1594.000000,1594.000000,1594.000000,1594.000000,1594.000000,1594.000000,1594.000000,1594.0,1594.000000,1594.00000
mean,2.675031,0.281681,0.160602,9.527604,1.028231,4.899780,0.643582,16.282329,30.615692,0.0,0.658720,0.46675
std,1.094891,0.449960,0.367279,0.712759,0.400666,2.893338,0.658182,8.309081,14.506806,0.0,0.474288,0.49905
min,2.000000,0.000000,0.000000,7.000000,0.000000,0.220000,0.010000,4.064020,6.969437,0.0,0.000000,0.00000
25%,2.000000,0.000000,0.000000,9.000000,1.000000,2.922500,0.270000,10.898822,20.215483,0.0,0.000000,0.00000
50%,2.000000,0.000000,0.000000,10.000000,1.000000,4.240000,0.440000,13.887867,27.467574,0.0,1.000000,0.00000
75%,3.000000,1.000000,0.000000,10.000000,1.000000,6.150000,0.720000,19.492885,38.228041,0.0,1.000000,1.00000
max,6.000000,1.000000,1.000000,10.000000,2.000000,16.320000,4.600000,49.072867,80.032507,0.0,1.000000,1.00000


In [16]:
x_train_const = sm.add_constant(x_train) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.533
Model:                            OLS   Adj. R-squared:                  0.530
Method:                 Least Squares   F-statistic:                     164.5
Date:                Thu, 09 Mar 2023   Prob (F-statistic):          1.47e-252
Time:                        22:01:56   Log-Likelihood:                -9383.7
No. Observations:                1594   AIC:                         1.879e+04
Df Residuals:                    1582   BIC:                         1.886e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                -91.2408     33

In [10]:
sts = StandardScaler()

x_train_standarized = sts.fit_transform(x_train)
x_test_standarized = sts.transform(x_test)

x_train_standarized = pd.DataFrame(x_train_standarized)

In [11]:
x_train_const = sm.add_constant(x_train_standarized.to_numpy()) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test_standarized) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.530
Model:                            OLS   Adj. R-squared:                  0.528
Method:                 Least Squares   F-statistic:                     223.3
Date:                Wed, 08 Mar 2023   Prob (F-statistic):          2.05e-253
Time:                        12:16:50   Log-Likelihood:                -9389.9
No. Observations:                1594   AIC:                         1.880e+04
Df Residuals:                    1585   BIC:                         1.885e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        231.7922      2.198    105.445      0.0

In [12]:
lm = LinearRegression()
model = lm.fit(x_train,y_train)
model.coef_
model.intercept_

-72.0380439935588

In [13]:
model.score(x_train, y_train)

0.529894582155652

In [14]:
y_pred = pd.DataFrame(model.predict(x_test),columns = ['realSum'] )      # model prediction
y_pred_train =  pd.DataFrame(model.predict(x_train),columns = ['realSum'])

In [15]:
mse(y_test,y_pred)

6151.578864363072

In [16]:
rmse = np.sqrt(mse(y_test,y_pred))
rmse

78.43200153230231

In [17]:
##prediction on the train set
mse(y_train,y_pred_train)

7658.991608390198

In [18]:
mae(y_test,y_pred)

54.97888924313564

In [19]:
R2=r2_score(y_test,y_pred)
R2

0.5163798730701148

In [20]:
R2_test=model.score(x_test,y_test)
R2_train=model.score(x_train,y_train)
Adj_R2= 1 - (1-R2)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)
Adj_R2

0.5080415950195996

In [21]:
model_lasso=Lasso(alpha=0.05)

model_lasso.fit(x_train, y_train)
print(f"{model_lasso.__class__.__name__}: Train -> {model_lasso.score(x_train, y_train)}, Test -> {model_lasso.score(x_test, y_test)}")

Lasso: Train -> 0.529888969091943, Test -> 0.5165577721139522


In [22]:
model_ridge=Ridge(alpha=1000)
model_ridge.fit(x_train, y_train)
print(f"{model_ridge.__class__.__name__}: Train -> {model_ridge.score(x_train, y_train)}, Test -> {model_ridge.score(x_test, y_test)}")

Ridge: Train -> 0.4151978444731774, Test -> 0.3964370856128421


In [23]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

print("X_train has %d rows and %d columns"  %(x_train.shape[0],x_train.shape[1]))
print("-----------------------------------")
print("The coefficient of determination for the test data is R2=%.2f"
      %(model.score(x_test, y_test)))
print("The coefficient of determination for the train data is R2=%.2f"
      %(model.score(x_train, y_train)))


X_train has 1594 rows and 9 columns
-----------------------------------
The coefficient of determination for the test data is R2=0.43
The coefficient of determination for the train data is R2=1.00


In [24]:
scores=cross_val_score(model, x_train, y_train, cv=5)
print("Cross validation scores: ", scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Cross validation scores:  [0.15921903 0.34135309 0.14296481 0.43194715 0.55013445]
Score stats: 0.33 accuracy with a standard deviation of 0.16


In [25]:
model1 = DecisionTreeRegressor()
model2 = LinearRegression()
model3 = KNeighborsRegressor()

model_pipeline = [model1, model2, model3]
model_names = ['Regression Tree', 'Linear Regression', 'KNN']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, x_train, y_train, cv=5))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

{'Regression Tree': 0.3338922311948313, 'Linear Regression': 0.5172681426048967, 'KNN': 0.4384480184753746}


In [26]:
data_berlin_cleaned.to_excel('data_berlin.xlsx')